# Exercise 4: Transient multifield problems

## Contents

We will learn

 - How to solve transient multi-field PDEs in Gridap

## Problem statement

We consider the transient Stokes equations.

Assuming the solution of the problem is

$$
\begin{aligned}
u(x,t) &= t(x_1,x_2)^T, \\
p(x,t) &= t(x_1-x_2),
\end{aligned}
$$

the PDE we want to solve is:

$$
\left\lbrace
\begin{aligned}
\frac{\partial u(t)}{\partial t} - \Delta u(t) + \nabla p &= f(t) & \text{ in }\Omega, \\
\nabla \cdot u(t) &= g(t) & \text{ in } \Omega, \\
u(x,t) &= t(x_1,x_2)^T & \text{ on } \partial\Omega, \\
u(x,0) &= 0.0 & \text{ in } \Omega, \\
p(x,0) &= 0.0 & \text{ in } \Omega,
\end{aligned}
\right.
$$

with $\Omega \doteq (0,1)^d$, $d=2$, and $f$ and $g$ are the source terms to be computed from the expressions of $u$ and $p$.

## Numerical scheme

### Approximation in space

We choose inf-sub stable $\boldsymbol{Q}_2/Q_{1}$ FEs.

The weak form of the problem reads: find $(u,p) \in U_h(t) \times Q_0$ such that

$$
  m(t,(u,p),(v,q)) + a(t,(u,p),(v,q)) = b(t,(v,q)) \quad \forall (v,q) \in \ V_h \times Q_0
$$

with $U_h(t)$ is a transient FE space and

$$
\begin{aligned}
m(t,(u,p),(v,q)) &= \int_\Omega \frac{\partial u}{\partial t} v \ d\Omega, \\
a(t,(u,p),(v,q)) &= \int_{\Omega} \nabla u \cdot \nabla v \ {\rm d}\Omega - \int_{\Omega} (\nabla\cdot v) \ p \ {\rm d}\Omega + \int_{\Omega} q \ (\nabla \cdot u) \ {\rm d}\Omega, \\
b(t,(v,q)) &= \int_\Omega f(t) \cdot v \ d\Omega + \int_\Omega g(t) \ q \ d\Omega.
\end{aligned}
$$

### Approximation in space

We will consider the ThetaMethod.

## Create the discrete model

### Exercise 1

_Load Gridap and create a $50\times50$ Cartesian grid of the unit square._

## Set  up multifield FE spaces

### Exercise 2

_Create the test FE spaces of the problem._

The spaces of test functions are constant in time and are defined as in steady problems.

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 For the velocities, we need to create the standard vector-valued continuous Lagrangian test FE space of order $k$. For the pressures, the standard scalar-valued continuous Lagrangian test FE space of order $k-1$ with zero mean value. We choose k = 2.

 Use the tag `boundary` to set up Dirichlet BCs for the velocity everywhere on the boundary $\partial \Omega$.
</details>

The trial space for `u` is a `TransientTrialFESpace`.

In [ ]:
u(x,t::Real) = t*VectorValue(x[1],x[2]) # Used, e.g., in visualisation
u(t::Real) = x -> u(x,t)                # Used to compute time derivative

U = TransientTrialFESpace(V,u)

Meanwhile, the space for `p` is constant in time.

In [ ]:
P = TrialFESpace(Q)

### Exercise 3

_Create the multifield FE spaces._

The trial multifield FE space must be transient (`U` is transient).

_Can you guess the name of a transient multifield FE space signature?_

## Triangulation and integration quadrature

In [ ]:
degree = 2*order
Ωₕ = Interior(model)
dΩ = Measure(Ωₕ,degree)

## Defining the source terms `f(t)` and `g(t)`

We have that

$$
f(x,t) = \frac{\partial u(x,t)}{\partial t} - \Delta u(x,t) + \nabla p = (x_1+t,x_2-t)^T \quad \text{and} \quad g(x,t) = 2t.
$$

We define `f(x,t)` and `g(x,t)` in two equivalent ways.

First, in a conventional manner:

### Exercise 4

_Write the expressions for `f` and `g` as written above._

Like `u` before, they must be time-dependent functions that return a space-only function.

Alternatively, we can use AD to get `f` and `g`.

In [ ]:
p(t::Real) = x -> t*(x[1]-x[2])

f_AD(t::Real) = x -> ∂t(u)(t)(x)-Δ(u(t))(x)+ ∇(p(t))(x)
g_AD(t::Real) = x -> (∇⋅u(t))(x)

We can do some quick (non-exhaustive) checks:

In [ ]:
@assert f(1.0)(Point(0.5,0.5)) == f_AD(1.0)(Point(0.5,0.5)) == VectorValue(1.5,-0.5)
@assert g(1.0)(Point(0.5,0.5)) == g_AD(1.0)(Point(0.5,0.5)) == 2.0

## Write down the weak form and the FE operator of the problem

Since the problem is linear, we use the `TransientAffineFEOperator` signature.

We handle time-dependent quantities by passing $t$ as additional argument to the forms.

Meanwhile, for the time derivative we define $m$ as a "steady-state" mass contribution.

### Exercise 5

_Write the bilinear forms `m`, `a` and the time-dependent linear form `b`._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 - Recall that $m$ is expressed as a "steady-state" mass contribution.
 - The only variables needing an explicit time dependency are the source terms `f` and `g`.
</details>

$$
\begin{aligned}
m(t,(u,p),(v,q)) &= \int_\Omega \frac{\partial u}{\partial t} v \ d\Omega, \\
a(t,(u,p),(v,q)) &= \int_{\Omega} \nabla u \cdot \nabla v \ {\rm d}\Omega - \int_{\Omega} (\nabla\cdot v) \ p \ {\rm d}\Omega + \int_{\Omega} q \ (\nabla \cdot u) \ {\rm d}\Omega, \\
b(t,(v,q)) &= \int_\Omega f(t) \cdot v \ d\Omega + \int_\Omega g(t) \ q \ d\Omega.
\end{aligned}
$$

Hence, we instantiate the `TransientAffineFEOperator` as

In [ ]:
op = TransientAffineFEOperator(m,a,b,X,Y)

## Set up the transient FE solver

We use `LUSolver` to solve the system at each time step.

In [ ]:
ls = LUSolver()

As ODE solver, we use the 2nd order `ThetaMethod` ($\theta = 0.5$).

### Exercise 6

_Define the ODE solver. Use a `ThetaMethod` with `ls` as the solver, $dt = 0.05$ and $\theta = 0.5$._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 - Use `methods(ThetaMethod)` to get the signature of the `ThetaMethod` constructor.
 - Note that the `nls` variable in the constructor is a `NonlinearSolver` and `LinearSolver <: NonlinearSolver`.
</details>

## Initial condition

In [ ]:
u₀ = interpolate_everywhere(u(0.0),U(0.0))
p₀ = interpolate_everywhere(p(0.0),P)
x₀ = interpolate_everywhere([u₀,p₀],X(0.0))

## Transient solver

In [ ]:
t₀ = 0.0
T = 1.0
xₕₜ = solve(ode_solver,op,x₀,t₀,T)

**Question:** _Did you expect a longer time during the `solve` execution?_

`xₕₜ` is just an _iterable_ function and the results at each $t$ are only computed when iterating over it

## Postprocessing

We compute and output the problem solutions:

In [ ]:
createpvd("transient_stokes_solution") do pvd
  for (xₕ,t) in xₕₜ
    (uₕ,pₕ) = xₕ
    pvd[t] = createvtk(Ωₕ,"transient_stokes_solution_$t"*".vtu",cellfields=["u"=>uₕ,"p"=>pₕ])
  end
end

And visualise them in ParaView.

## References

[Gridap Tutorial 17: Transient Poisson Equation](https://gridap.github.io/Tutorials/dev/pages/t017_poisson_transient/#Tutorial-17:-Transient-Poisson-equation-1)

### Bonus exercises

1. _The solution of this problem belongs to the FE space, since it is linear in space and time. Hence the FE solutions `uₕ` and `pₕ` should coincide with `u` and `p` at every time step. Compute the l2 norm of the errors for `uₕ` and `pₕ` inside the iteration over `xₕₜ` and check exacteness (up to arithmetical precision errors)._
2. _Write down the residual of the problem and solve it defining the operator as `op = TransientFEOperator(res,X,Y)`, i.e. with the Jacobian computed with automatic differentiation. You might find help on how to do this in [Gridap Tutorial 17: Transient Poisson Equation](https://gridap.github.io/Tutorials/dev/pages/t017_poisson_transient/#Tutorial-17:-Transient-Poisson-equation-1)._

**Exercise done!**

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*